In [2]:
import speech_recognition as sr
import pickle
import glob
import asr.align

In [3]:
transDict = dict()
count = 0
for filename in glob.iglob('/datasets/home/35/335/cs253wgf/291/Data/dev-clean/**/*.trans.txt', recursive=True):
    with open(filename) as f:
        for line in f:
            vals = line.split(" ", 1)
            transDict[vals[0]] = vals[1].lower()
    count += 1
print(count)
print(len(transDict))
pickle.dump(transDict, open("transDict.p", "wb"))

97
2703


In [9]:
r = sr.Recognizer()
def getTextFromAudio(filename):
    with sr.AudioFile(filename) as source:
        audio = r.record(source)
    audioText = ""
    try:
        audioText = r.recognize_google(audio)
        return audioText
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
 
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))

In [10]:
count = 0
WER = 0
speechRecConvertDict = dict()
for filename in glob.iglob('/datasets/home/35/335/cs253wgf/291/Data/dev-clean/**/*.wav', recursive=True):
    vals = filename.rsplit("/", 1)
    audioFileName = (vals[1].split(".", 1))[0]
    text1 = transDict[audioFileName]
    text2 = getTextFromAudio(filename)
    speechRecConvertDict[audioFileName] = text2
    if(not text1):
        text1 = ""
    if(not text2):
        text2 = ""
    WER = WER + asr.align.calculate_wer(text1, text2)
    count += 1
    if(count%10==0):
        print(WER)
print("Done")
print("Total count", count)
print("Average WER:", WER/count)

0.3677286872269407
1.151361661986286
1.6680337582048566
2.545071077480633
3.2296477441118934
3.9536549922586843
5.2691042728816155
6.221316291366865
7.7200003259558905
9.234347979307039
10.350730145965336
11.416761221973998
12.419589104867583
12.80425029900648
13.749723052968939
14.57135042687666
14.978471205955831
15.929995518538673
16.658706644301326
17.964343076341635
18.52649141256604
19.678618400889658
20.368245372092634
20.824586715714094
21.582015090592495
22.19785632011113
23.0800251026137
24.06285497480041
25.171138507456018
25.9191839624684
27.32206361205686
29.031795411014894
30.01161835203286
30.436879150158152
31.094008511786676
31.38620272632773
32.027369801020114
32.67337665083859
33.406150062084066
34.0295729229513
35.17863348121942
35.76043126216896
36.27805530688764
37.029970130538324
37.67074303399831
38.00013632088432
38.743487451221874
39.30109778304115
39.8191216584286
40.61140819643327
41.07144106233277
41.82709755607897
42.7201105132307
43.132118087243136
43.762

In [11]:
pickle.dump(speechRecConvertDict, open("speechRecConvertDict.p", "wb"))

In [15]:
speechFromGoog = pickle.load(open("speechRecConvertDict.p", "rb"))
orignSpeech = pickle.load(open("transDict.p", "rb"))

In [19]:
WER = 0
for s in speechFromGoog:
    if(speechFromGoog[s]):
        WER = WER + asr.align.calculate_wer(speechFromGoog[s].split(), orignSpeech[s].split())
#         print(WER)
print(WER/len(speechFromGoog))

0.21402795528478075


In [32]:
asr.align.calculate_wer("is there".split(), "who is there".split())

0.5